This notebook is an example of interacting with iManage system.
It re-downloads document files for each Document previously imported from iManage.

In [ ]:
import os
import io
import csv
import sys
import uuid
sys.path.append('../../')
os.environ["DJANGO_SETTINGS_MODULE"] = "settings"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings")
import django
from django.db import connection, transaction
from django.conf import settings
django.setup()
from apps.document.models import Document
from apps.imanage_integration.models import IManageDocument, IManageConfig
from django.utils.text import get_valid_filename
from django.core.files.storage import FileSystemStorage
from apps.common.streaming_utils import download_file

In [ ]:
for imanage_config in IManageConfig.objects.all():
    auth_token = imanage_config.login()
    for imanage_doc in IManageDocument.objects.filter(imanage_config=imanage_config):
        doc = imanage_doc.document
        if not doc:
            print(f'Imanage doc #{imanage_doc.id}: No Document assigned. Ignoring.')
            continue
            
        filename, response = imanage_config.load_document(auth_token, imanage_doc.imanage_doc_id)
        upload_session_id = str(uuid.uuid4())
        filename = get_valid_filename(filename)

        dst_folder = os.path.join(
            settings.MEDIA_ROOT,
            settings.FILEBROWSER_DOCUMENTS_DIRECTORY,
            upload_session_id)
        os.makedirs(dst_folder, exist_ok=True)

        dst_folder_storage = FileSystemStorage(location=dst_folder)
        filepath = dst_folder_storage.path(filename)

        download_file(response, filepath)
        
        document_name = filename
        document_source = upload_session_id
        doc.name = document_name
        doc.source = document_source
        doc.save(update_fields=['name', 'source'])
        print(f'Imanage doc #{imanage_doc.id}: Downloaded file: {filepath}. Updated document #{doc.id}.')
        
        